In [ ]:



# ===============================
# 1️⃣ IMPORT LIBRARIES
# ===============================
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
import numpy as np
import os

print("✅ TensorFlow Version:", tf.__version__)

# ===============================
# 2️⃣ DATA PREPARATION
# ===============================
# Replace with your dataset path (e.g., /content/waste_dataset/)
base_dir = "/content/waste_dataset"

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

print("✅ Classes found:", train_generator.class_indices)

# ===============================
# 3️⃣ MODEL ARCHITECTURE (CNN)
# ===============================
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 classes
])

model.summary()

# ===============================
# 4️⃣ COMPILE THE MODEL
# ===============================
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ===============================
# 5️⃣ TRAIN THE MODEL
# ===============================
epochs = 5  # increase later for better accuracy
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)

# ===============================
# 6️⃣ PLOT ACCURACY & LOSS
# ===============================
plt.figure(figsize=(8, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy', marker='o')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='o')
plt.title('📈 Model Accuracy per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Training Loss', marker='o')
plt.plot(history.history['val_loss'], label='Validation Loss', marker='o')
plt.title('📉 Model Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# ===============================
# 7️⃣ SAVE MODEL
# ===============================
model.save("waste_classifier.h5")
print("✅ Model saved as waste_classifier.h5")

# ===============================
# 8️⃣ SAMPLE PREDICTION (OPTIONAL)
# ===============================
# Upload a sample image using the left panel (Files) in Colab or code below:
# from google.colab import files
# uploaded = files.upload()

# Example: test image path
# test_image_path = '/content/test_plastic.jpg'
# from tensorflow.keras.preprocessing import image

# img = image.load_img(test_image_path, target_size=(150,150))
# img_array = image.img_to_array(img)/255.0
# img_array = np.expand_dims(img_array, axis=0)

# prediction = model.predict(img_array)
# class_names = list(train_generator.class_indices.keys())
# predicted_class = class_names[np.argmax(prediction[0])]
# confidence = round(np.max(prediction[0]) * 100, 2)

# print(f"🧠 Predicted: {predicted_class} ({confidence}% confidence)")
